# TFM

## Importación de librerías

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import toml
import random
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix

## Ejecución de código en GPU

In [105]:
import tensorflow as tf

# Comprobar la versión de TensorFlow
print("TensorFlow version:", tf.__version__)

# Listar todos los dispositivos físicos disponibles
physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):")
    for gpu in physical_devices:
        print(f'\t{gpu}')
else:
    print("🔴 No se ha detectado ninguna GPU. TensorFlow está utilizando la CPU.")

TensorFlow version: 2.10.0
🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):
	PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Lectura de fichero

In [106]:
data_dict = {}
DATA_DIR = '../../data/'

for data_folder in os.listdir(DATA_DIR):
    *data_files, config_file = os.listdir(f'{DATA_DIR}/{data_folder}')

    leak_value = toml.load(f'{DATA_DIR}/{data_folder}/{config_file}')['tanks']['flow_value']

    train_dataframe, test_data = random.sample(data_files, 2)

    data_dict[data_folder] = {
        'leak_value': leak_value,
        'train_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{train_dataframe}').drop(columns=['Unnamed: 0']),
        'test_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{test_data}').drop(columns=['Unnamed: 0'])
    }

### Conjuntos de datos de entrenamiento

In [107]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"14217,87",-6025,"8078,98","16271,85","16202,83","-69,02","29,76",0
1,2,"16202,83",-5915,"7993,5","18281,33","18287,13","5,8","103,63",0
2,3,"18287,13",-6005,"5997,67","18279,8","18294,11","14,31","27,16",0
3,4,"18294,11",-6085,"3013,11","15222,22","15202,29","-19,93","93,76",0
4,5,"15202,29",-5800,"7941,86","17344,15","17431,85","87,7","159,23",0
...,...,...,...,...,...,...,...,...,...
995,996,"15406,65",-6030,"4978,48","14355,13","14337,52","-17,61","61,26",1
996,997,"14337,52",-6000,"5721,51","14059,03","14009,35","-49,68","-44,29",1
997,998,"14009,35",-6275,"6313,19","14047,54","14181,99","134,45","161,32",1
998,999,"14181,99",-6150,"5914,01","13946,0","14007,4","61,4","181,88",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22430,06","-6075,0","5976,32","22331,38","22376,0","44,62","-116,83",0
1,2,"22376,0","-6065,0","3965,07","20276,07","20290,81","14,74","-45,27",0
2,3,"20290,81","-5775,0","6982,55","21498,36","21513,87","15,51","-60,91",0
3,4,"21513,87","-6035,0","6956,14","22435,01","22426,74","-8,27","5,59",0
4,5,"22426,74","-6030,0","6377,94","22774,68","22817,15","42,47","36,97",0
...,...,...,...,...,...,...,...,...,...
995,996,"2107,2","-5828,98","6463,64","2741,86","2620,44","-121,42","-167,54",1
996,997,"2620,44","-5955,0","6142,6","2808,04","2771,02","-37,02","-143,07",1
997,998,"2771,02","-5896,0","3108,82","-16,16","25,62","41,78","-150,11",1
998,999,"25,62","-5592,26","6421,37","854,73","896,61","41,88","-83,97",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23654,87","-6005,0","4298,93","21948,8","21972,61","23,81","163,31",0
1,2,"21972,61","-6095,0","4622,96","20500,57","20617,61","117,04","322,76",0
2,3,"20617,61","-6175,0","9067,79","23510,4","23442,51","-67,89","221,84",0
3,4,"23442,51","-5990,0","3410,32","20862,83","20890,7","27,87","233,22",0
4,5,"20890,7","-5915,0","6810,38","21786,08","21803,84","17,76","215,53",0
...,...,...,...,...,...,...,...,...,...
995,996,"10,65","-0,0","0,0","10,65","30,77","20,12","26,54",1
996,997,"30,77","-0,0","0,0","30,77","0,47","-30,3","-3,4",1
997,998,"0,47","-0,0","0,0","0,47","4,08","3,61","0,87",1
998,999,"4,08","-0,0","0,0","4,08","1,82","-2,26","-3,71",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21399,05","-5790,0","5638,02","21247,07","21227,09","-19,98","63,76",0
1,2,"21227,09","-5895,0","5241,74","20573,83","20576,89","3,06","133,61",0
2,3,"20576,89","-6055,0","6999,43","21521,32","21504,09","-17,23","33,97",0
3,4,"21504,09","-5995,0","9004,73","24513,82","24667,5","153,68","232,55",0
4,5,"24667,5","-6005,0","5447,75","24110,25","24146,71","36,46","202,77",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,62","-0,0","0,0","2,62","19,24","16,62","5,27",1
996,997,"19,24","-0,0","0,0","19,24","14,88","-4,36","5,99",1
997,998,"14,88","-0,0","0,0","14,88","9,37","-5,51","-4,99",1
998,999,"9,37","-0,0","0,0","9,37","3,89","-5,48","-11,62",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"19617,1","-5720,0","6040,65","19937,75","19908,69","-29,06","287,97",0
1,2,"19908,69","-6040,0","9876,76","23745,45","23727,8","-17,65","238,88",0
2,3,"23727,8","-6040,0","6042,62","23730,42","23778,29","47,87","259,99",0
3,4,"23778,29","-5985,0","4048,45","21841,74","21813,92","-27,82","123,38",0
4,5,"21813,92","-6055,0","6875,76","22634,68","22664,82","30,14","231,15",0
...,...,...,...,...,...,...,...,...,...
995,996,"10,5","-0,0","0,0","10,5","23,41","12,91","9,18",1
996,997,"23,41","-0,0","0,0","23,41","0,69","-22,72","-2,91",1
997,998,"0,69","-0,0","0,0","0,69","6,88","6,19","3,32",1
998,999,"6,88","-0,0","0,0","6,88","7,0","0,12","0,49",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16942,12","-5825,0","5491,42","16608,54","16673,87","65,33","187,29",0
1,2,"16673,87","-5825,0","6897,1","17745,97","17875,86","129,89","237,48",0
2,3,"17875,86","-5965,0","6950,25","18861,11","18894,66","33,55","253,19",0
3,4,"18894,66","-5970,0","7962,56","20887,22","20937,92","50,7","387,03",0
4,5,"20937,92","-6135,0","5924,27","20727,19","20818,82","91,63","502,42",0
...,...,...,...,...,...,...,...,...,...
995,996,"19,68","-0,0","0,0","19,68","8,25","-11,43","0,72",1
996,997,"8,25","-0,0","0,0","8,25","3,65","-4,6","-8,36",1
997,998,"3,65","-0,0","0,0","3,65","8,4","4,75","4,92",1
998,999,"8,4","-0,0","0,0","8,4","23,49","15,09","16,35",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"18109,75","-5940,0","6999,66","19169,41","19140,47","-28,94","290,5",0
1,2,"19140,47","-6320,0","7014,09","19834,56","19825,75","-8,81","346,46",0
2,3,"19825,75","-6145,0","7026,85","20707,6","20677,99","-29,61","326,59",0
3,4,"20677,99","-5805,0","4783,79","19656,78","19702,96","46,18","340,89",0
4,5,"19702,96","-6195,0","6592,28","20100,24","20021,69","-78,55","117,34",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,39","-0,0","0,0","3,39","8,94","5,55","-9,07",1
996,997,"8,94","-0,0","0,0","8,94","13,46","4,52","1,87",1
997,998,"13,46","-0,0","0,0","13,46","5,2","-8,26","4,92",1
998,999,"5,2","-0,0","0,0","5,2","13,18","7,98","-1,52",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16777,0","-6100,0","9895,09","20572,09","20589,64","17,55","-18,03",0
1,2,"20589,64","-6115,0","4955,9","19430,54","19468,61","38,07","-12,02",0
2,3,"19468,61","-5965,0","6946,41","20450,02","20512,05","62,03","14,91",0
3,4,"20512,05","-6245,0","2980,24","17247,29","17265,48","18,19","87,81",0
4,5,"17265,48","-6425,0","8878,71","19719,19","19875,47","156,28","275,63",0
...,...,...,...,...,...,...,...,...,...
995,996,"0,33","-0,0","0,0","0,33","0,14","-0,19","-6,95",1
996,997,"0,14","-0,0","0,0","0,14","10,7","10,56","-1,35",1
997,998,"10,7","-0,0","0,0","10,7","11,2","0,5","4,17",1
998,999,"11,2","-0,0","0,0","11,2","11,98","0,78","-1,3",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17698,38","-5710,0","4618,64","16607,02","16580,51","-26,51","35,66",0
1,2,"16580,51","-5875,0","6004,3","16709,81","16702,31","-7,5","58,61",0
2,3,"16702,31","-6250,0","8001,28","18453,59","18456,14","2,55","-9,07",0
3,4,"18456,14","-5870,0","6931,83","19517,97","19595,08","77,11","-16,24",0
4,5,"19595,08","-6330,0","7449,42","20714,5","20644,03","-70,47","-108,89",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,11","-0,0","0,0","1,11","1,53","0,42","-12,95",1
996,997,"1,53","-0,0","0,0","1,53","5,18","3,65","0,97",1
997,998,"5,18","-0,0","0,0","5,18","9,61","4,43","9,4",1
998,999,"9,61","-0,0","0,0","9,61","13,39","3,78","3,06",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16880,67","-6340,0","9520,82","20061,49","20089,2","27,71","239,98",0
1,2,"20089,2","-5965,0","8981,18","23105,38","23114,65","9,27","236,52",0
2,3,"23114,65","-6350,0","6284,23","23048,88","23080,01","31,13","96,76",0
3,4,"23080,01","-5495,0","3705,04","21290,05","21253,69","-36,36","173,15",0
4,5,"21253,69","-6010,0","6906,8","22150,49","22243,33","92,84","144,0",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,33","-0,0","0,0","2,33","8,66","6,33","4,22",1
996,997,"8,66","-0,0","0,0","8,66","8,69","0,03","3,5",1
997,998,"8,69","-0,0","0,0","8,69","21,02","12,33","20,09",1
998,999,"21,02","-0,0","0,0","21,02","5,64","-15,38","3,39",1


### Conjunto de datos de prueba

In [108]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21950,34",-6035,"7006,85","22922,19","22962,99","40,8","172,2",0
1,2,"22962,99",-6105,"6515,76","23373,75","23300,26","-73,49","125,79",0
2,3,"23300,26",-5770,"6438,69","23968,95","24020,7","51,75","97,38",0
3,4,"24020,7",-5915,"5427,88","23533,58","23536,74","3,16","79,51",0
4,5,"23536,74",-6415,"4351,1","21472,84","21473,49","0,65","87,66",0
...,...,...,...,...,...,...,...,...,...
995,996,"15927,66",-6095,"6450,17","16282,83","16239,24","-43,59","114,14",1
996,997,"16239,24",-5855,"6019,11","16403,35","16385,84","-17,51","38,4",1
997,998,"16385,84",-6120,"3750,83","14016,67","13979,49","-37,18","-21,93",1
998,999,"13979,49",-5845,"6725,82","14860,31","14857,03","-3,28","10,77",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21149,68","-5960,0","5940,46","21130,14","21204,78","74,64","255,0",0
1,2,"21204,78","-6315,0","7021,73","21911,51","21862,93","-48,58","164,23",0
2,3,"21862,93","-5890,0","3218,08","19191,01","19194,42","3,41","149,26",0
3,4,"19194,42","-6160,0","6739,31","19773,73","19802,52","28,79","127,48",0
4,5,"19802,52","-5960,0","6979,79","20822,31","20853,99","31,68","148,24",0
...,...,...,...,...,...,...,...,...,...
995,996,"2228,36","-5465,58","4141,29","904,07","873,35","-30,72","-173,3",1
996,997,"873,35","-6090,0","6592,68","1376,03","1220,09","-155,94","-308,8",1
997,998,"1220,09","-5980,0","7181,83","2421,92","2463,95","42,03","-206,86",1
998,999,"2463,95","-5850,0","5758,71","2372,66","2400,72","28,06","-176,26",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20423,93","-5590,0","5813,6","20647,53","20665,41","17,88","31,64",0
1,2,"20665,41","-5970,0","5997,78","20693,19","20680,86","-12,33","79,51",0
2,3,"20680,86","-5930,0","7951,68","22702,54","22757,13","54,59","150,95",0
3,4,"22757,13","-6260,0","6680,1","23177,23","23197,64","20,41","107,33",0
4,5,"23197,64","-6075,0","5816,09","22938,73","22983,34","44,61","181,99",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,36","-0,0","0,0","2,36","14,55","12,19","2,84",1
996,997,"14,55","-0,0","0,0","14,55","4,86","-9,69","-1,12",1
997,998,"4,86","-0,0","0,0","4,86","0,46","-4,4","0,26",1
998,999,"0,46","-0,0","0,0","0,46","11,22","10,76","-9,79",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21287,51","-6070,0","7042,91","22260,42","22216,71","-43,71","188,2",0
1,2,"22216,71","-6065,0","4682,57","20834,28","20856,74","22,46","259,79",0
2,3,"20856,74","-6035,0","7234,09","22055,83","22134,3","78,47","272,27",0
3,4,"22134,3","-6120,0","5847,84","21862,14","22001,79","139,65","380,74",0
4,5,"22001,79","-5605,0","3757,23","20154,02","20186,77","32,75","279,92",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,31","-0,0","0,0","3,31","4,22","0,91","-5,75",1
996,997,"4,22","-0,0","0,0","4,22","14,3","10,08","7,99",1
997,998,"14,3","-0,0","0,0","14,3","12,66","-1,64","10,33",1
998,999,"12,66","-0,0","0,0","12,66","2,96","-9,7","-1,54",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20535,61","-5940,0","8641,12","23236,73","23294,75","58,02","265,22",0
1,2,"23294,75","-6575,0","6455,26","23175,01","23296,05","121,04","387,15",0
2,3,"23296,05","-5630,0","6600,92","24266,97","24325,7","58,73","366,14",0
3,4,"24325,7","-6015,0","3190,31","21501,01","21523,46","22,45","375,09",0
4,5,"21523,46","-5695,0","7376,83","23205,29","23238,07","32,78","376,68",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,69","-0,0","0,0","3,69","23,8","20,11","18,39",1
996,997,"23,8","-0,0","0,0","23,8","15,45","-8,35","12,43",1
997,998,"15,45","-0,0","0,0","15,45","10,85","-4,6","-0,6",1
998,999,"10,85","-0,0","0,0","10,85","3,49","-7,36","-0,14",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,65","-6055,0","6215,36","22304,01","22242,32","-61,69","34,11",0
1,2,"22242,32","-5940,0","6691,65","22993,97","22983,66","-10,31","42,41",0
2,3,"22983,66","-5885,0","6130,98","23229,64","23301,88","72,24","58,48",0
3,4,"23301,88","-6030,0","5035,77","22307,65","22270,88","-36,77","25,47",0
4,5,"22270,88","-6005,0","6518,15","22784,03","22794,42","10,39","13,96",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,74","-0,0","0,0","2,74","3,88","1,14","3,0",1
996,997,"3,88","-0,0","0,0","3,88","0,72","-3,16","-2,72",1
997,998,"0,72","-0,0","0,0","0,72","1,13","0,41","-11,02",1
998,999,"1,13","-0,0","0,0","1,13","2,14","1,01","-19,34",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15886,17","-5900,0","5970,3","15956,47","15980,01","23,54","68,46",0
1,2,"15980,01","-5810,0","6050,74","16220,75","16175,65","-45,1","8,64",0
2,3,"16175,65","-5760,0","5917,51","16333,16","16413,96","80,8","17,3",0
3,4,"16413,96","-5970,0","6982,8","17426,76","17460,02","33,26","68,23",0
4,5,"17460,02","-6340,0","7929,64","19049,66","19111,41","61,75","52,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,52","-0,0","0,0","5,52","0,47","-5,05","-4,84",1
996,997,"0,47","-0,0","0,0","0,47","5,79","5,32","0,68",1
997,998,"5,79","-0,0","0,0","5,79","14,14","8,35","-0,25",1
998,999,"14,14","-0,0","0,0","14,14","4,73","-9,41","2,66",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22465,51","-6080,0","7789,55","24175,06","24208,87","33,81","-175,32",0
1,2,"24208,87","-6045,0","4948,93","23112,8","23139,14","26,34","-178,39",0
2,3,"23139,14","-5935,0","7036,17","24240,31","24285,57","45,26","-129,43",0
3,4,"24285,57","-5950,0","4812,44","23148,01","23215,86","67,85","74,7",0
4,5,"23215,86","-6310,0","5513,33","22419,19","22370,85","-48,34","67,2",0
...,...,...,...,...,...,...,...,...,...
995,996,"13,88","-0,0","0,0","13,88","5,95","-7,93","1,78",1
996,997,"5,95","-0,0","0,0","5,95","12,1","6,15","10,02",1
997,998,"12,1","-0,0","0,0","12,1","5,06","-7,04","-9,98",1
998,999,"5,06","-0,0","0,0","5,06","4,17","-0,89","-3,22",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23299,15","-5900,0","4046,57","21445,72","21419,26","-26,46","496,31",0
1,2,"21419,26","-6310,0","8439,67","23548,93","23527,4","-21,53","356,96",0
2,3,"23527,4","-6055,0","7093,15","24565,55","24485,13","-80,42","233,55",0
3,4,"24485,13","-5925,0","2956,05","21516,18","21525,33","9,15","216,75",0
4,5,"21525,33","-6095,0","6018,92","21449,25","21425,77","-23,48","121,03",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,16","-0,0","0,0","3,16","2,82","-0,34","-5,42",1
996,997,"2,82","-0,0","0,0","2,82","2,05","-0,77","1,14",1
997,998,"2,05","-0,0","0,0","2,05","1,39","-0,66","-3,8",1
998,999,"1,39","-0,0","0,0","1,39","10,38","8,99","-2,67",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"11544,4","-5565,0","7069,47","13048,87","12982,38","-66,49","118,8",0
1,2,"12982,38","-6155,0","6953,56","13780,94","13840,48","59,54","124,42",0
2,3,"13840,48","-6005,0","8211,32","16046,8","15937,63","-109,17","-18,99",0
3,4,"15937,63","-5915,0","8550,06","18572,69","18595,78","23,09","76,99",0
4,5,"18595,78","-5830,0","6389,87","19155,65","19072,47","-83,18","-24,06",0
...,...,...,...,...,...,...,...,...,...
995,996,"16,15","-0,0","0,0","16,15","6,29","-9,86","-11,87",1
996,997,"6,29","-0,0","0,0","6,29","14,36","8,07","4,38",1
997,998,"14,36","-0,0","0,0","14,36","2,38","-11,98","-20,25",1
998,999,"2,38","-0,0","0,0","2,38","4,75","2,37","-6,32",1


## Limpieza de datos

In [109]:
bad_formatted_fields = [
    'Volumen dep. almacenam. ini. (L)',
    'Llenado dep. almacenam. (L)',
    'Volumen dep. almacenam. fin. teor. (L)',
    'Volumen dep. almacenam. fin. (L)',
    'Variacion',
    'Variacion Acum.',
]

for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        for field in bad_formatted_fields:
            data_dict[case][dataset][field] = data_dict[case][dataset][field].str.replace(',', '.')
            data_dict[case][dataset][field] = pd.to_numeric(data_dict[case][dataset][field])

### Conjunto de datos de entrenamiento

In [110]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,14217.87,-6025,8078.98,16271.85,16202.83,-69.02,29.76,0
1,2,16202.83,-5915,7993.50,18281.33,18287.13,5.80,103.63,0
2,3,18287.13,-6005,5997.67,18279.80,18294.11,14.31,27.16,0
3,4,18294.11,-6085,3013.11,15222.22,15202.29,-19.93,93.76,0
4,5,15202.29,-5800,7941.86,17344.15,17431.85,87.70,159.23,0
...,...,...,...,...,...,...,...,...,...
995,996,15406.65,-6030,4978.48,14355.13,14337.52,-17.61,61.26,1
996,997,14337.52,-6000,5721.51,14059.03,14009.35,-49.68,-44.29,1
997,998,14009.35,-6275,6313.19,14047.54,14181.99,134.45,161.32,1
998,999,14181.99,-6150,5914.01,13946.00,14007.40,61.40,181.88,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22430.06,"-6075,0",5976.32,22331.38,22376.00,44.62,-116.83,0
1,2,22376.00,"-6065,0",3965.07,20276.07,20290.81,14.74,-45.27,0
2,3,20290.81,"-5775,0",6982.55,21498.36,21513.87,15.51,-60.91,0
3,4,21513.87,"-6035,0",6956.14,22435.01,22426.74,-8.27,5.59,0
4,5,22426.74,"-6030,0",6377.94,22774.68,22817.15,42.47,36.97,0
...,...,...,...,...,...,...,...,...,...
995,996,2107.20,"-5828,98",6463.64,2741.86,2620.44,-121.42,-167.54,1
996,997,2620.44,"-5955,0",6142.60,2808.04,2771.02,-37.02,-143.07,1
997,998,2771.02,"-5896,0",3108.82,-16.16,25.62,41.78,-150.11,1
998,999,25.62,"-5592,26",6421.37,854.73,896.61,41.88,-83.97,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23654.87,"-6005,0",4298.93,21948.80,21972.61,23.81,163.31,0
1,2,21972.61,"-6095,0",4622.96,20500.57,20617.61,117.04,322.76,0
2,3,20617.61,"-6175,0",9067.79,23510.40,23442.51,-67.89,221.84,0
3,4,23442.51,"-5990,0",3410.32,20862.83,20890.70,27.87,233.22,0
4,5,20890.70,"-5915,0",6810.38,21786.08,21803.84,17.76,215.53,0
...,...,...,...,...,...,...,...,...,...
995,996,10.65,"-0,0",0.00,10.65,30.77,20.12,26.54,1
996,997,30.77,"-0,0",0.00,30.77,0.47,-30.30,-3.40,1
997,998,0.47,"-0,0",0.00,0.47,4.08,3.61,0.87,1
998,999,4.08,"-0,0",0.00,4.08,1.82,-2.26,-3.71,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21399.05,"-5790,0",5638.02,21247.07,21227.09,-19.98,63.76,0
1,2,21227.09,"-5895,0",5241.74,20573.83,20576.89,3.06,133.61,0
2,3,20576.89,"-6055,0",6999.43,21521.32,21504.09,-17.23,33.97,0
3,4,21504.09,"-5995,0",9004.73,24513.82,24667.50,153.68,232.55,0
4,5,24667.50,"-6005,0",5447.75,24110.25,24146.71,36.46,202.77,0
...,...,...,...,...,...,...,...,...,...
995,996,2.62,"-0,0",0.00,2.62,19.24,16.62,5.27,1
996,997,19.24,"-0,0",0.00,19.24,14.88,-4.36,5.99,1
997,998,14.88,"-0,0",0.00,14.88,9.37,-5.51,-4.99,1
998,999,9.37,"-0,0",0.00,9.37,3.89,-5.48,-11.62,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,19617.10,"-5720,0",6040.65,19937.75,19908.69,-29.06,287.97,0
1,2,19908.69,"-6040,0",9876.76,23745.45,23727.80,-17.65,238.88,0
2,3,23727.80,"-6040,0",6042.62,23730.42,23778.29,47.87,259.99,0
3,4,23778.29,"-5985,0",4048.45,21841.74,21813.92,-27.82,123.38,0
4,5,21813.92,"-6055,0",6875.76,22634.68,22664.82,30.14,231.15,0
...,...,...,...,...,...,...,...,...,...
995,996,10.50,"-0,0",0.00,10.50,23.41,12.91,9.18,1
996,997,23.41,"-0,0",0.00,23.41,0.69,-22.72,-2.91,1
997,998,0.69,"-0,0",0.00,0.69,6.88,6.19,3.32,1
998,999,6.88,"-0,0",0.00,6.88,7.00,0.12,0.49,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16942.12,"-5825,0",5491.42,16608.54,16673.87,65.33,187.29,0
1,2,16673.87,"-5825,0",6897.10,17745.97,17875.86,129.89,237.48,0
2,3,17875.86,"-5965,0",6950.25,18861.11,18894.66,33.55,253.19,0
3,4,18894.66,"-5970,0",7962.56,20887.22,20937.92,50.70,387.03,0
4,5,20937.92,"-6135,0",5924.27,20727.19,20818.82,91.63,502.42,0
...,...,...,...,...,...,...,...,...,...
995,996,19.68,"-0,0",0.00,19.68,8.25,-11.43,0.72,1
996,997,8.25,"-0,0",0.00,8.25,3.65,-4.60,-8.36,1
997,998,3.65,"-0,0",0.00,3.65,8.40,4.75,4.92,1
998,999,8.40,"-0,0",0.00,8.40,23.49,15.09,16.35,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,18109.75,"-5940,0",6999.66,19169.41,19140.47,-28.94,290.50,0
1,2,19140.47,"-6320,0",7014.09,19834.56,19825.75,-8.81,346.46,0
2,3,19825.75,"-6145,0",7026.85,20707.60,20677.99,-29.61,326.59,0
3,4,20677.99,"-5805,0",4783.79,19656.78,19702.96,46.18,340.89,0
4,5,19702.96,"-6195,0",6592.28,20100.24,20021.69,-78.55,117.34,0
...,...,...,...,...,...,...,...,...,...
995,996,3.39,"-0,0",0.00,3.39,8.94,5.55,-9.07,1
996,997,8.94,"-0,0",0.00,8.94,13.46,4.52,1.87,1
997,998,13.46,"-0,0",0.00,13.46,5.20,-8.26,4.92,1
998,999,5.20,"-0,0",0.00,5.20,13.18,7.98,-1.52,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16777.00,"-6100,0",9895.09,20572.09,20589.64,17.55,-18.03,0
1,2,20589.64,"-6115,0",4955.90,19430.54,19468.61,38.07,-12.02,0
2,3,19468.61,"-5965,0",6946.41,20450.02,20512.05,62.03,14.91,0
3,4,20512.05,"-6245,0",2980.24,17247.29,17265.48,18.19,87.81,0
4,5,17265.48,"-6425,0",8878.71,19719.19,19875.47,156.28,275.63,0
...,...,...,...,...,...,...,...,...,...
995,996,0.33,"-0,0",0.00,0.33,0.14,-0.19,-6.95,1
996,997,0.14,"-0,0",0.00,0.14,10.70,10.56,-1.35,1
997,998,10.70,"-0,0",0.00,10.70,11.20,0.50,4.17,1
998,999,11.20,"-0,0",0.00,11.20,11.98,0.78,-1.30,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17698.38,"-5710,0",4618.64,16607.02,16580.51,-26.51,35.66,0
1,2,16580.51,"-5875,0",6004.30,16709.81,16702.31,-7.50,58.61,0
2,3,16702.31,"-6250,0",8001.28,18453.59,18456.14,2.55,-9.07,0
3,4,18456.14,"-5870,0",6931.83,19517.97,19595.08,77.11,-16.24,0
4,5,19595.08,"-6330,0",7449.42,20714.50,20644.03,-70.47,-108.89,0
...,...,...,...,...,...,...,...,...,...
995,996,1.11,"-0,0",0.00,1.11,1.53,0.42,-12.95,1
996,997,1.53,"-0,0",0.00,1.53,5.18,3.65,0.97,1
997,998,5.18,"-0,0",0.00,5.18,9.61,4.43,9.40,1
998,999,9.61,"-0,0",0.00,9.61,13.39,3.78,3.06,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16880.67,"-6340,0",9520.82,20061.49,20089.20,27.71,239.98,0
1,2,20089.20,"-5965,0",8981.18,23105.38,23114.65,9.27,236.52,0
2,3,23114.65,"-6350,0",6284.23,23048.88,23080.01,31.13,96.76,0
3,4,23080.01,"-5495,0",3705.04,21290.05,21253.69,-36.36,173.15,0
4,5,21253.69,"-6010,0",6906.80,22150.49,22243.33,92.84,144.00,0
...,...,...,...,...,...,...,...,...,...
995,996,2.33,"-0,0",0.00,2.33,8.66,6.33,4.22,1
996,997,8.66,"-0,0",0.00,8.66,8.69,0.03,3.50,1
997,998,8.69,"-0,0",0.00,8.69,21.02,12.33,20.09,1
998,999,21.02,"-0,0",0.00,21.02,5.64,-15.38,3.39,1


### Conjunto de datos de prueba

In [111]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21950.34,-6035,7006.85,22922.19,22962.99,40.80,172.20,0
1,2,22962.99,-6105,6515.76,23373.75,23300.26,-73.49,125.79,0
2,3,23300.26,-5770,6438.69,23968.95,24020.70,51.75,97.38,0
3,4,24020.70,-5915,5427.88,23533.58,23536.74,3.16,79.51,0
4,5,23536.74,-6415,4351.10,21472.84,21473.49,0.65,87.66,0
...,...,...,...,...,...,...,...,...,...
995,996,15927.66,-6095,6450.17,16282.83,16239.24,-43.59,114.14,1
996,997,16239.24,-5855,6019.11,16403.35,16385.84,-17.51,38.40,1
997,998,16385.84,-6120,3750.83,14016.67,13979.49,-37.18,-21.93,1
998,999,13979.49,-5845,6725.82,14860.31,14857.03,-3.28,10.77,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21149.68,"-5960,0",5940.46,21130.14,21204.78,74.64,255.00,0
1,2,21204.78,"-6315,0",7021.73,21911.51,21862.93,-48.58,164.23,0
2,3,21862.93,"-5890,0",3218.08,19191.01,19194.42,3.41,149.26,0
3,4,19194.42,"-6160,0",6739.31,19773.73,19802.52,28.79,127.48,0
4,5,19802.52,"-5960,0",6979.79,20822.31,20853.99,31.68,148.24,0
...,...,...,...,...,...,...,...,...,...
995,996,2228.36,"-5465,58",4141.29,904.07,873.35,-30.72,-173.30,1
996,997,873.35,"-6090,0",6592.68,1376.03,1220.09,-155.94,-308.80,1
997,998,1220.09,"-5980,0",7181.83,2421.92,2463.95,42.03,-206.86,1
998,999,2463.95,"-5850,0",5758.71,2372.66,2400.72,28.06,-176.26,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20423.93,"-5590,0",5813.60,20647.53,20665.41,17.88,31.64,0
1,2,20665.41,"-5970,0",5997.78,20693.19,20680.86,-12.33,79.51,0
2,3,20680.86,"-5930,0",7951.68,22702.54,22757.13,54.59,150.95,0
3,4,22757.13,"-6260,0",6680.10,23177.23,23197.64,20.41,107.33,0
4,5,23197.64,"-6075,0",5816.09,22938.73,22983.34,44.61,181.99,0
...,...,...,...,...,...,...,...,...,...
995,996,2.36,"-0,0",0.00,2.36,14.55,12.19,2.84,1
996,997,14.55,"-0,0",0.00,14.55,4.86,-9.69,-1.12,1
997,998,4.86,"-0,0",0.00,4.86,0.46,-4.40,0.26,1
998,999,0.46,"-0,0",0.00,0.46,11.22,10.76,-9.79,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21287.51,"-6070,0",7042.91,22260.42,22216.71,-43.71,188.20,0
1,2,22216.71,"-6065,0",4682.57,20834.28,20856.74,22.46,259.79,0
2,3,20856.74,"-6035,0",7234.09,22055.83,22134.30,78.47,272.27,0
3,4,22134.30,"-6120,0",5847.84,21862.14,22001.79,139.65,380.74,0
4,5,22001.79,"-5605,0",3757.23,20154.02,20186.77,32.75,279.92,0
...,...,...,...,...,...,...,...,...,...
995,996,3.31,"-0,0",0.00,3.31,4.22,0.91,-5.75,1
996,997,4.22,"-0,0",0.00,4.22,14.30,10.08,7.99,1
997,998,14.30,"-0,0",0.00,14.30,12.66,-1.64,10.33,1
998,999,12.66,"-0,0",0.00,12.66,2.96,-9.70,-1.54,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20535.61,"-5940,0",8641.12,23236.73,23294.75,58.02,265.22,0
1,2,23294.75,"-6575,0",6455.26,23175.01,23296.05,121.04,387.15,0
2,3,23296.05,"-5630,0",6600.92,24266.97,24325.70,58.73,366.14,0
3,4,24325.70,"-6015,0",3190.31,21501.01,21523.46,22.45,375.09,0
4,5,21523.46,"-5695,0",7376.83,23205.29,23238.07,32.78,376.68,0
...,...,...,...,...,...,...,...,...,...
995,996,3.69,"-0,0",0.00,3.69,23.80,20.11,18.39,1
996,997,23.80,"-0,0",0.00,23.80,15.45,-8.35,12.43,1
997,998,15.45,"-0,0",0.00,15.45,10.85,-4.60,-0.60,1
998,999,10.85,"-0,0",0.00,10.85,3.49,-7.36,-0.14,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.65,"-6055,0",6215.36,22304.01,22242.32,-61.69,34.11,0
1,2,22242.32,"-5940,0",6691.65,22993.97,22983.66,-10.31,42.41,0
2,3,22983.66,"-5885,0",6130.98,23229.64,23301.88,72.24,58.48,0
3,4,23301.88,"-6030,0",5035.77,22307.65,22270.88,-36.77,25.47,0
4,5,22270.88,"-6005,0",6518.15,22784.03,22794.42,10.39,13.96,0
...,...,...,...,...,...,...,...,...,...
995,996,2.74,"-0,0",0.00,2.74,3.88,1.14,3.00,1
996,997,3.88,"-0,0",0.00,3.88,0.72,-3.16,-2.72,1
997,998,0.72,"-0,0",0.00,0.72,1.13,0.41,-11.02,1
998,999,1.13,"-0,0",0.00,1.13,2.14,1.01,-19.34,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15886.17,"-5900,0",5970.30,15956.47,15980.01,23.54,68.46,0
1,2,15980.01,"-5810,0",6050.74,16220.75,16175.65,-45.10,8.64,0
2,3,16175.65,"-5760,0",5917.51,16333.16,16413.96,80.80,17.30,0
3,4,16413.96,"-5970,0",6982.80,17426.76,17460.02,33.26,68.23,0
4,5,17460.02,"-6340,0",7929.64,19049.66,19111.41,61.75,52.84,0
...,...,...,...,...,...,...,...,...,...
995,996,5.52,"-0,0",0.00,5.52,0.47,-5.05,-4.84,1
996,997,0.47,"-0,0",0.00,0.47,5.79,5.32,0.68,1
997,998,5.79,"-0,0",0.00,5.79,14.14,8.35,-0.25,1
998,999,14.14,"-0,0",0.00,14.14,4.73,-9.41,2.66,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22465.51,"-6080,0",7789.55,24175.06,24208.87,33.81,-175.32,0
1,2,24208.87,"-6045,0",4948.93,23112.80,23139.14,26.34,-178.39,0
2,3,23139.14,"-5935,0",7036.17,24240.31,24285.57,45.26,-129.43,0
3,4,24285.57,"-5950,0",4812.44,23148.01,23215.86,67.85,74.70,0
4,5,23215.86,"-6310,0",5513.33,22419.19,22370.85,-48.34,67.20,0
...,...,...,...,...,...,...,...,...,...
995,996,13.88,"-0,0",0.00,13.88,5.95,-7.93,1.78,1
996,997,5.95,"-0,0",0.00,5.95,12.10,6.15,10.02,1
997,998,12.10,"-0,0",0.00,12.10,5.06,-7.04,-9.98,1
998,999,5.06,"-0,0",0.00,5.06,4.17,-0.89,-3.22,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23299.15,"-5900,0",4046.57,21445.72,21419.26,-26.46,496.31,0
1,2,21419.26,"-6310,0",8439.67,23548.93,23527.40,-21.53,356.96,0
2,3,23527.40,"-6055,0",7093.15,24565.55,24485.13,-80.42,233.55,0
3,4,24485.13,"-5925,0",2956.05,21516.18,21525.33,9.15,216.75,0
4,5,21525.33,"-6095,0",6018.92,21449.25,21425.77,-23.48,121.03,0
...,...,...,...,...,...,...,...,...,...
995,996,3.16,"-0,0",0.00,3.16,2.82,-0.34,-5.42,1
996,997,2.82,"-0,0",0.00,2.82,2.05,-0.77,1.14,1
997,998,2.05,"-0,0",0.00,2.05,1.39,-0.66,-3.80,1
998,999,1.39,"-0,0",0.00,1.39,10.38,8.99,-2.67,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,11544.40,"-5565,0",7069.47,13048.87,12982.38,-66.49,118.80,0
1,2,12982.38,"-6155,0",6953.56,13780.94,13840.48,59.54,124.42,0
2,3,13840.48,"-6005,0",8211.32,16046.80,15937.63,-109.17,-18.99,0
3,4,15937.63,"-5915,0",8550.06,18572.69,18595.78,23.09,76.99,0
4,5,18595.78,"-5830,0",6389.87,19155.65,19072.47,-83.18,-24.06,0
...,...,...,...,...,...,...,...,...,...
995,996,16.15,"-0,0",0.00,16.15,6.29,-9.86,-11.87,1
996,997,6.29,"-0,0",0.00,6.29,14.36,8.07,4.38,1
997,998,14.36,"-0,0",0.00,14.36,2.38,-11.98,-20.25,1
998,999,2.38,"-0,0",0.00,2.38,4.75,2.37,-6.32,1


<!-- TODO -->

## Selección de características

In [112]:
for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        data_dict[case][dataset] = data_dict[case][dataset].filter(items=['Variacion', 'Fugando combustible'])

### Conjunto de datos de entrenamiento

In [113]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Variacion,Fugando combustible
0,-69.02,0
1,5.80,0
2,14.31,0
3,-19.93,0
4,87.70,0
...,...,...
995,-17.61,1
996,-49.68,1
997,134.45,1
998,61.40,1


,Variacion,Fugando combustible
0,44.62,0
1,14.74,0
2,15.51,0
3,-8.27,0
4,42.47,0
...,...,...
995,-121.42,1
996,-37.02,1
997,41.78,1
998,41.88,1


,Variacion,Fugando combustible
0,23.81,0
1,117.04,0
2,-67.89,0
3,27.87,0
4,17.76,0
...,...,...
995,20.12,1
996,-30.30,1
997,3.61,1
998,-2.26,1


,Variacion,Fugando combustible
0,-19.98,0
1,3.06,0
2,-17.23,0
3,153.68,0
4,36.46,0
...,...,...
995,16.62,1
996,-4.36,1
997,-5.51,1
998,-5.48,1


,Variacion,Fugando combustible
0,-29.06,0
1,-17.65,0
2,47.87,0
3,-27.82,0
4,30.14,0
...,...,...
995,12.91,1
996,-22.72,1
997,6.19,1
998,0.12,1


...

,Variacion,Fugando combustible
0,65.33,0
1,129.89,0
2,33.55,0
3,50.70,0
4,91.63,0
...,...,...
995,-11.43,1
996,-4.60,1
997,4.75,1
998,15.09,1


,Variacion,Fugando combustible
0,-28.94,0
1,-8.81,0
2,-29.61,0
3,46.18,0
4,-78.55,0
...,...,...
995,5.55,1
996,4.52,1
997,-8.26,1
998,7.98,1


,Variacion,Fugando combustible
0,17.55,0
1,38.07,0
2,62.03,0
3,18.19,0
4,156.28,0
...,...,...
995,-0.19,1
996,10.56,1
997,0.50,1
998,0.78,1


,Variacion,Fugando combustible
0,-26.51,0
1,-7.50,0
2,2.55,0
3,77.11,0
4,-70.47,0
...,...,...
995,0.42,1
996,3.65,1
997,4.43,1
998,3.78,1


,Variacion,Fugando combustible
0,27.71,0
1,9.27,0
2,31.13,0
3,-36.36,0
4,92.84,0
...,...,...
995,6.33,1
996,0.03,1
997,12.33,1
998,-15.38,1


### Conjunto de datos de prueba

In [114]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Variacion,Fugando combustible
0,40.80,0
1,-73.49,0
2,51.75,0
3,3.16,0
4,0.65,0
...,...,...
995,-43.59,1
996,-17.51,1
997,-37.18,1
998,-3.28,1


,Variacion,Fugando combustible
0,74.64,0
1,-48.58,0
2,3.41,0
3,28.79,0
4,31.68,0
...,...,...
995,-30.72,1
996,-155.94,1
997,42.03,1
998,28.06,1


,Variacion,Fugando combustible
0,17.88,0
1,-12.33,0
2,54.59,0
3,20.41,0
4,44.61,0
...,...,...
995,12.19,1
996,-9.69,1
997,-4.40,1
998,10.76,1


,Variacion,Fugando combustible
0,-43.71,0
1,22.46,0
2,78.47,0
3,139.65,0
4,32.75,0
...,...,...
995,0.91,1
996,10.08,1
997,-1.64,1
998,-9.70,1


,Variacion,Fugando combustible
0,58.02,0
1,121.04,0
2,58.73,0
3,22.45,0
4,32.78,0
...,...,...
995,20.11,1
996,-8.35,1
997,-4.60,1
998,-7.36,1


...

,Variacion,Fugando combustible
0,-61.69,0
1,-10.31,0
2,72.24,0
3,-36.77,0
4,10.39,0
...,...,...
995,1.14,1
996,-3.16,1
997,0.41,1
998,1.01,1


,Variacion,Fugando combustible
0,23.54,0
1,-45.10,0
2,80.80,0
3,33.26,0
4,61.75,0
...,...,...
995,-5.05,1
996,5.32,1
997,8.35,1
998,-9.41,1


,Variacion,Fugando combustible
0,33.81,0
1,26.34,0
2,45.26,0
3,67.85,0
4,-48.34,0
...,...,...
995,-7.93,1
996,6.15,1
997,-7.04,1
998,-0.89,1


,Variacion,Fugando combustible
0,-26.46,0
1,-21.53,0
2,-80.42,0
3,9.15,0
4,-23.48,0
...,...,...
995,-0.34,1
996,-0.77,1
997,-0.66,1
998,8.99,1


,Variacion,Fugando combustible
0,-66.49,0
1,59.54,0
2,-109.17,0
3,23.09,0
4,-83.18,0
...,...,...
995,-9.86,1
996,8.07,1
997,-11.98,1
998,2.37,1


<!-- TODO -->

## Unificación de conjunto de datos

In [115]:
complete_train_data = pd.concat([element['train_dataframe'] for element in data_dict.values()])

train_data = (
    complete_train_data.drop(columns='Fugando combustible'), 
    complete_train_data['Fugando combustible']
)

In [116]:
segmented_test_data = [
    (element['leak_value'], element['test_dataframe']) for element in data_dict.values()
]

test_data = [(
    'general',
    pd.concat([element['test_dataframe'] for element in data_dict.values()], ignore_index=True)
)]

## Implementación de modelos

### Definición de métricas

In [117]:
def leak_detection_delay(y_true: np.array, y_pred: np.array) -> int:
    real_first_leak_day = 0
    while y_true[real_first_leak_day] != 1: real_first_leak_day += 1

    pred_first_leak_day = real_first_leak_day
    while y_pred[pred_first_leak_day] != 1: pred_first_leak_day += 1

    return (pred_first_leak_day - real_first_leak_day)

### Modelos clásicos

In [118]:
from sklearn.model_selection import GridSearchCV
import pickle as pkl

param_grids_path = '../../pkl/param_grids'

def train_classic_model(classifier_class, param_grid, train_data, cv = 5):
    param_pkl_path = f'{param_grids_path}/{classifier_class.__name__}.pkl'

    classifier = None

    X_train, y_train = train_data

    if os.path.exists(param_pkl_path):  # Param grid already known
        param_grid = pkl.load(open(param_pkl_path, 'rb'))
        classifier = classifier_class(**param_grid)
        classifier.fit(X_train, y_train)
    else:
        grid_search = GridSearchCV(
            estimator = classifier_class(),
            param_grid = param_grid,
            cv = cv
        )
        grid_search.fit(X_train, y_train)
        classifier = grid_search.best_estimator_
        pkl.dump(
            obj = grid_search.best_params_,
            file = open(param_pkl_path, 'wb')
        )
    
    return classifier

In [119]:
def test_classic_model(classifier, test_list):
    results = list()

    for (leak_value, test_data) in test_list:

        X_test = test_data.drop(columns='Fugando combustible')
        y_test = test_data['Fugando combustible']

        y_pred = classifier.predict(X_test)

        results.append((
            leak_value, 
            accuracy_score(y_test, y_pred),
            f1_score(y_test, y_pred),
            leak_detection_delay(y_test, y_pred),
            (leak_value * 60 * 24) * leak_detection_delay(y_test, y_pred) if type(leak_value) == float else 0
        ))
    
    return pd.DataFrame.from_records(
        data = results,
        columns = [
            'leak value',
            'accuracy',
            'f1 score',
            'leak detection delay (days)',
            'leak loss (liters)'
        ]
    )

##### Random Forest Classifier

In [120]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = train_classic_model(RandomForestClassifier, rf_param_grid, train_data)

In [121]:
display(test_classic_model(rf_model, segmented_test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,0.013000,0.490,0.359296,2,37.440000
1,0.054929,0.540,0.430693,1,79.097143
2,0.096857,0.635,0.610459,0,0.000000
3,0.138786,0.719,0.720398,1,199.851429
4,0.180714,0.763,0.764647,1,260.228571
5,0.222643,0.779,0.795937,0,0.000000
6,0.264571,0.787,0.801860,1,380.982857
7,0.306500,0.828,0.838649,0,0.000000
8,0.348429,0.804,0.820841,0,0.000000
9,0.390357,0.805,0.823529,0,0.000000


In [122]:
display(test_classic_model(rf_model, test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,general,0.751133,0.758928,2,0


##### Naive Bayes Classifier

In [123]:
from sklearn.naive_bayes import GaussianNB

gnb_param_grid = {
    'var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

gnb_model = train_classic_model(GaussianNB, gnb_param_grid, train_data)

In [124]:
display(test_classic_model(gnb_model, segmented_test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,0.013000,0.509,0.295552,0,0.000000
1,0.054929,0.608,0.494845,1,79.097143
2,0.096857,0.606,0.494872,1,139.474286
3,0.138786,0.582,0.442667,1,199.851429
4,0.180714,0.556,0.403226,0,0.000000
5,0.222643,0.530,0.352617,0,0.000000
6,0.264571,0.522,0.324859,0,0.000000
7,0.306500,0.517,0.294891,0,0.000000
8,0.348429,0.508,0.280702,0,0.000000
9,0.390357,0.510,0.262048,0,0.000000


In [125]:
display(test_classic_model(gnb_model, test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,general,0.525067,0.321395,0,0


##### KNN Classifier

In [126]:
from sklearn.neighbors import KNeighborsClassifier

knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_model = train_classic_model(KNeighborsClassifier, knn_param_grid, train_data)

In [127]:
display(test_classic_model(knn_model, segmented_test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,0.013000,0.494,0.373762,5,93.600000
1,0.054929,0.550,0.456522,1,79.097143
2,0.096857,0.627,0.606955,0,0.000000
3,0.138786,0.711,0.713009,1,199.851429
4,0.180714,0.747,0.751229,1,260.228571
5,0.222643,0.776,0.793358,0,0.000000
6,0.264571,0.770,0.786245,0,0.000000
7,0.306500,0.812,0.823640,0,0.000000
8,0.348429,0.772,0.793103,0,0.000000
9,0.390357,0.789,0.809739,0,0.000000


In [128]:
display(test_classic_model(knn_model, test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,general,0.740133,0.749647,5,0


##### XGBoost Classifier

In [129]:
from xgboost import XGBClassifier

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

xgb_model = train_classic_model(XGBClassifier, xgb_param_grid, train_data)

In [130]:
display(test_classic_model(xgb_model, segmented_test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,0.013000,0.495,0.366374,2,37.440000
1,0.054929,0.530,0.419753,1,79.097143
2,0.096857,0.635,0.607104,0,0.000000
3,0.138786,0.735,0.736842,1,199.851429
4,0.180714,0.761,0.764995,1,260.228571
5,0.222643,0.782,0.798148,0,0.000000
6,0.264571,0.797,0.811513,1,380.982857
7,0.306500,0.836,0.848429,0,0.000000
8,0.348429,0.806,0.821691,0,0.000000
9,0.390357,0.815,0.833483,0,0.000000


In [131]:
display(test_classic_model(xgb_model, test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,general,0.7566,0.764891,2,0


##### _Ensemble Classifier_

In [132]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier()),
        ('XGB', XGBClassifier())
    ],
    voting='hard'
)

voting_clf.fit(train_data[0], train_data[1])

VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('nb', GaussianNB()),
                             ('knn', KNeighborsClassifier()),
                             ('XGB',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))])

In [133]:
display(test_classic_model(voting_clf, segmented_test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,0.013000,0.487,0.313253,2,37.440000
1,0.054929,0.541,0.403121,1,79.097143
2,0.096857,0.626,0.576923,0,0.000000
3,0.138786,0.695,0.676564,1,199.851429
4,0.180714,0.735,0.718982,1,260.228571
5,0.222643,0.744,0.739308,0,0.000000
6,0.264571,0.756,0.755020,0,0.000000
7,0.306500,0.789,0.787084,0,0.000000
8,0.348429,0.746,0.741344,0,0.000000
9,0.390357,0.760,0.759036,0,0.000000


In [134]:
display(test_classic_model(voting_clf, test_data))

,leak value,accuracy,f1 score,leak detection delay (days),leak loss (liters)
0,general,0.7162,0.701242,2,0


### Modelos _Deep Learning_

In [ ]:
def eval_dl_model(model, test_list):
    result_dict = {}
    for leak_value, test_dataframe in test_list:
        X_test = test_dataframe.drop(columns='Fugando combustible')
        y_test = test_dataframe['Fugando combustible']
        
        y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

        accuracy = accuracy_score(y_test, y_pred)
        leak_delay = leak_detection_delay(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        leak_loss = (leak_value * 60 * 24) * leak_delay

        result_dict[leak_value] = (accuracy, f1, leak_delay, leak_loss)

    return pd.DataFrame.from_dict(
        data = result_dict,
        orient = 'index',
        columns = ['accuracy_score', 'f1_score', 'leak_detection_delay', 'leak_loss']
    )

#### Único modelo de _Deep Learning_

In [ ]:
X_train, y_train = train_data

##### Modelo unicapa

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(15, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

display(eval_dl_model(model, segmented_test_data))

##### Modelo multicapa

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(25, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(40, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(60, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

display(eval_dl_model(model, segmented_test_data))

#### Múltiples modelos de _Deep Learning_

In [ ]:
results = {}
for case in data_dict:
    X_train = data_dict[case]['train_dataframe'].drop(columns='Fugando combustible')
    y_train = data_dict[case]['train_dataframe']['Fugando combustible']
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = 50,
        batch_size = 8192,
        verbose = 0
    )

    leak_value = data_dict[case]['leak_value']

    X_test = data_dict[case]['test_dataframe'].drop(columns='Fugando combustible')
    y_test = data_dict[case]['test_dataframe']['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    leak_delay = leak_detection_delay(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results[leak_value] = (accuracy, f1, leak_delay)
    # model.save(f'../../pkl/models/{case}.keras')

display(pd.DataFrame.from_dict(
    data = results,
    orient = 'index',
    columns = ['accuracy_score', 'f1_score', 'leak_detection_delay']
))